In [1]:
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm

In [2]:
poblacion = pd.read_csv('../Data/clean_data/densidad_poblacion.csv')

In [3]:
poblacion.head()

,prov_id,mun_id,total_2023,hombres_2023,mujeres_2023,ine_id,municipio,km2,densidad
0,1,1001,2975.0,1539.0,1436.0,1001000000,Alegria-dulantzi,1995.0,1.491228
1,1,1002,10313.0,5134.0,5179.0,1002000000,Amurrio,9635.0,1.070368
2,1,1003,1409.0,732.0,677.0,1003000000,Aramaio,7327.0,0.192302
3,1,1004,1832.0,898.0,934.0,1004000000,Artziniega,2729.0,0.671308
4,1,1006,232.0,120.0,112.0,1006000000,Armiñon,1294.0,0.179289


In [4]:
coord = pd.read_excel('../Data/int/listado-longitud-latitud-municipios-espana.xls')

In [14]:
coord.drop(columns=['Comunidad','Provincia'], inplace = True)
coord.columns = coord.columns.str.lower()
coord.rename(columns={'población':'municipio'}, inplace = True)

KeyError: "['Comunidad', 'Provincia'] not found in axis"

In [15]:
coord['municipio'] = coord['municipio'].str.capitalize()
coord.head()

,municipio,latitud,longitud
0,Abla,37.14114,-2.780104
1,Abrucena,37.13305,-2.797098
2,Adra,36.74807,-3.022522
3,Albanchez,37.28710,-2.181163
4,Alboloduy,37.03319,-2.621750


In [16]:
replace = {'á':'a', 'à':'a',
           'é':'e', 'è':'e',
           'í':'i', 'ì':'i', 
           'ó':'o', 'ò':'o', 'ô':'o', 
           'ú':'u', 
           'Á':'A', 'À':'A',
           'Ó':'O',
           'Ò':'O',
           'Ú':'U', 
           'É':'E', 
           'Ì':'I',
           'Í':'I',
           r'\([^\)]*\)':''
          }
coord.replace(replace, regex=True, inplace = True)

In [7]:
poblacion.head()

,prov_id,mun_id,total_2023,hombres_2023,mujeres_2023,ine_id,municipio,km2,densidad
0,1,1001,2975.0,1539.0,1436.0,1001000000,Alegria-dulantzi,1995.0,1.491228
1,1,1002,10313.0,5134.0,5179.0,1002000000,Amurrio,9635.0,1.070368
2,1,1003,1409.0,732.0,677.0,1003000000,Aramaio,7327.0,0.192302
3,1,1004,1832.0,898.0,934.0,1004000000,Artziniega,2729.0,0.671308
4,1,1006,232.0,120.0,112.0,1006000000,Armiñon,1294.0,0.179289


In [17]:
density_loc = pd.merge(poblacion, coord, how='outer', on='municipio')

In [18]:
density_loc['latitud'].isnull().sum()

750

In [19]:
density_loc['total_2023'].isnull().sum()

724

In [20]:
#usamos la librería fuzzywuzzy con las poblaciones que no tienen correlación:
original = density_loc[density_loc['latitud'].isnull()]['municipio'].tolist()   #lista que mantenemos
cambiar = density_loc[density_loc['total_2023'].isnull()]['municipio'].tolist() #lista a modificar

In [21]:
from fuzzywuzzy import process

# Listas de nombres de ciudades
#cambiar   # Lista que queremos modificar
#original  # Lista del INE

# Comparar cada ciudad de cities_list1 con todas las ciudades de cities_list2 y reemplazar si la similitud es alta
threshold = 80  
 

replacements = {}
for c in tqdm(cambiar):
    match = process.extractOne(c, original)
    if match[1] >= threshold:
        if c not in replacements:
            replacements[c] = match[0]

  0%|          | 0/724 [00:00<?, ?it/s]

In [13]:
replacements

{'Alcudia de Monteagud': 'Alcudia de monteagud',
 'Alhama de Almeria': 'Alhama de almeria',
 'Armuña de Almanzora': 'Armuña de almanzora',
 'Castro de Filabres': 'Castro de filabres',
 'Cuevas del Almanzora': 'Cuevas del almanzora',
 'Ejido ': 'El ejido',
 'Gallardos ': 'Los gallardos',
 'Huercal-Overa': 'Huercal-overa',
 'Huercal de Almeria': 'Huercal de almeria',
 'Laujar de Andarax': 'Laujar de andarax',
 'Lucainena de las Torres': 'Lucainena de las torres',
 'Mojonera ': 'La mojonera',
 'Olula de Castro': 'Olula de castro',
 'Olula del Rio': 'Olula del rio',
 'Paterna del Rio': 'Paterna del rio',
 'Roquetas de Mar': 'Roquetas de mar',
 'Santa Cruz de Marchena': 'Santa cruz de marchena',
 'Santa Fe de Mondujar': 'Santa fe de mondujar',
 'Tres Villas ': 'Las tres villas',
 'Uleila del Campo': 'Uleila del campo',
 'Velez-Blanco': 'Velez-blanco',
 'Velez-Rubio': 'Velez-rubio',
 'Alcala de los Gazules': 'Alcala de los gazules',
 'Alcala del Valle': 'Alcala del valle',
 'Arcos de la Fron